<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ds-kiel/TinyML-Labs/tree/WS23-24/Lab1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ds-kiel/TinyML-Labs/tree/WS23-24/Lab1.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/ds-kiel/TinyML-Labs/tree/WS23-24/Lab1.ipynb" download><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

---


Before starting, you must click on the "Copy To Drive" option in the top bar. Go to File --> Save a Copy to Drive. Name it *'Group\<Your group number\>_Lab1.ipynb'*. <ins>This is the master notebook so you will not be able to save your changes without copying it !</ins> Once you click on that, make sure you are working on that version of the notebook so that your work is saved.



---

# Lab 1: Machine Learning with Tensorflow

Before we can execute a machine learning model on a microcontroller, we first need to create the model, and train it. Especially the training part is quite resource intensive and thus (these days) needs to be performed on a more powerful machine before deployment on a resource-constrained device like a microcontroller.

For a start and to get everyone ready for the next steps, you will explore in this lab how to train a neural network with Tensorflow/Keras. Moreover, you will use the [Edge Impulse Python SDK](https://docs.edgeimpulse.com/docs/tools/edge-impulse-python-sdk) to estimate the RAM, ROM, and inference time for your models on the [target platform](https://store.arduino.cc/products/arduino-tiny-machine-learning-kit) which we are going to use for the next labs.

In part 1 of this lab, you will explore or recap some machine learning characteristics using the [MNIST dataset](https://www.tensorflow.org/datasets/catalog/mnist), whereas in part 2 you build your own neural networks using either the [Fashion MNIST](https://www.tensorflow.org/datasets/catalog/fashion_mnist) dataset or the [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

## Environment

The instructions for this lab come as a [Jupyter Notebook](https://jupyter.org/). You can run it locally in your own Python environment, but we recommend you to use [Google Colab](https://colab.research.google.com) to save your computer hardware, have an instantly working python environment, and allow for easy collaboration.

Moreover, you need to obtain an API key from an Edge Impulse project. Register at [edgeimpulse.com](https://edgeimpulse.com/), log in and create a new project. Open the project, navigate to **Dashboard** and click on the **Keys** tab to view your API keys. Double-click on the API key to highlight it, right-click, and select **Copy**. Paste the key below in the cell starting with `ei.API_KEY`.

![Copy API key from Edge Impulse project](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/python-sdk-copy-ei-api-key.png)

For this lab you will not use the project in the Edge Impulse Studio. We just need the API Key.

## What do you need to hand in?

This Jupyter Notebook is intended as a document that you use both for working on the lab as well as for answering the questions. For handing in your lab, please **upload this Jupyter notebook** as well as **a PDF version** of it. Make sure that all images you include are visible in the PDF version.

In [ ]:
# If you have not done so already, install the following dependencies
!python -m pip install tensorflow scikit-learn edgeimpulse numpy matplotlib seaborn

### Imports

In [ ]:
import numpy as np
import pandas as pd

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import edgeimpulse as ei

import matplotlib.pyplot as plt
import seaborn as sns

### Helper Functions

In [ ]:
plt.style.use('seaborn-v0_8')

def plot_training_history(history, model_name):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(f'Model {model_name}')
    fig.set_figwidth(15)

    ax1.plot(range(1, len(history.history['accuracy'])+1), history.history['accuracy'])
    ax1.plot(range(1, len(history.history['val_accuracy'])+1), history.history['val_accuracy'])
    ax1.set_title('Model accuracy')
    ax1.set(xlabel='epoch', ylabel='accuracy')
    ax1.legend(['training', 'validation'], loc='best')

    ax2.plot(range(1, len(history.history['loss'])+1), history.history['loss'])
    ax2.plot(range(1, len(history.history['val_loss'])+1), history.history['val_loss'])
    ax2.set_title('Model loss')
    ax2.set(xlabel='epoch', ylabel='loss')
    ax2.legend(['training', 'validation'], loc='best')
    plt.show()

Paste your Edge Impulse API key string in the `ei.API_KEY` value in the following cell:

In [ ]:
ei.API_KEY = "ei_dae2..." # Change this to your Edge Impulse API key

## Part 1: Tensorflow and MNIST

In this part, you will use the standard dataset for getting started with machine learning, so to say the "Hello World" of machine learning: MNIST. MNIST is a dataset consisting of 70.000 grayscale images of handwritten digits with a resolution of 28x28 pixels.

![Example of MNIST images](https://storage.googleapis.com/tfds-data/visualization/fig/mnist-3.0.1.png)

Your task is to run the following models, explore the influence of the number of epochs you use for training, breifly explain the models, and set the models into perspective to each other regarding resource consumption on the *Arduino Nano 33 BLE* (the same MCU used as in the [Arduino Tiny Machine Learning Kit](https://store.arduino.cc/products/arduino-tiny-machine-learning-kit)).

### Prepare the Data

In [ ]:
# Model / data parameters
labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
num_classes = len(labels)
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

---

**Task 1:** Briefly explain the cell above.

**Answer:**

---

### Build the models

We build three models:

- `model_1`: fully connected neural network with a single hidden dense layer
- `model_2`: Convolutional Neural Network (CNN)
- `model_3`: Combination of a CNN and a dense layer  

In [ ]:
# Build model_1
def build_model_1(summary=False):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model_1
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    if summary:
        model.summary()

    return model

In [ ]:
# Build model_2
def build_model_2(summary=False):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model_2
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    if summary:
        model.summary()

    return model

In [ ]:
# Build model_3
def build_model_3(summary=False):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', input_shape=input_shape))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model_3
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    if summary:
        model.summary()

    return model

In [ ]:
model_1 = build_model_1(True)
keras.utils.plot_model(model_1, show_shapes=True)

In [ ]:
model_2 = build_model_2(True)
keras.utils.plot_model(model_2, show_shapes=True)

In [ ]:
model_3 = build_model_3(True)
keras.utils.plot_model(model_3, show_shapes=True)

---

**Task 2:** Explain the three models.

**<ins>Answer</ins>**

Model 1: ...

Model 2: ...

Model 3: ...

---

### Train and evaluate model 1

**Please note:** When you want to retrain a model, you need to create a new model. Otherwise, you will continue training your already trained model.

#### **Train model 1**

In [ ]:
model_1_short = build_model_1()
history_1_short = model_1_short.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.1)
plot_training_history(history_1_short, 1)

In [ ]:
model_1_long = build_model_1()
history_1_long = model_1_long.fit(x_train, y_train, batch_size=128, epochs=100, validation_split=0.1)
plot_training_history(history_1_long, 1)

So far we trained our first model once for 5 for once for 100 epochs. But which is the better option? Is more episodes always better?

---
**Task 3:** Explain the parameters, we used in our fitting function (`fit(x_train, y_train, batch_size=128, epochs=100, validation_split=0.1)`).

**Answer:**

**Taks 4:** Which behavior do you observe when training for 5 or for 100 episodes? Especially have a look at the plots.

**Answer:**

**Task 5:** Why does the validation loss start to increase again when we train for many epochs while the training loss continues to decrease?

**Answer:**

---

#### **Evaluate model 1**

**5 epochs:**

In [ ]:
score_model_1_short = model_1_short.evaluate(x_test, y_test) #, verbose=0)
print("Test loss (5 epochs):", score_model_1_short[0])
print("Test accuracy (5 epochs):", score_model_1_short[1])

cm = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(model_1_short.predict(x_test),axis=1))
# print(cm)

cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

cm = pd.DataFrame(cm, index = labels,
                  columns = labels)

plt.figure(figsize = (4,4))
ax = sns.heatmap(cm*100,
           annot=True,
           fmt='.1f',
           cmap="Blues",
           cbar=False,
              )
ax.set_ylabel("True Class", fontdict= {'fontweight':'bold'})
ax.set_xlabel("Predicted Class", fontdict= {'fontweight':'bold'})

plt.show()

**100 epochs:**

In [ ]:
score_model_1_long = model_1_long.evaluate(x_test, y_test) #, verbose=0)
print("Test loss (100 epochs):", score_model_1_long[0])
print("Test accuracy (100 epochs):", score_model_1_long[1])

cm = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(model_1_long.predict(x_test),axis=1))
# print(cm)

cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

cm = pd.DataFrame(cm, index = labels,
                  columns = labels)

plt.figure(figsize = (4,4))
ax = sns.heatmap(cm*100,
           annot=True,
           fmt='.1f',
           cmap="Blues",
           cbar=False,
              )
ax.set_ylabel("True Class", fontdict= {'fontweight':'bold'})
ax.set_xlabel("Predicted Class", fontdict= {'fontweight':'bold'})

plt.show()

---
**Task 6:** After evaluating the two trained models, which one is better?

**Answer:**

**Task 7:** Now retrain and evaluate the model and try to find the optimal number of epochs. How many epochs are necessary? How do you know that this is the correct number? *(You can use the cell right below to build your model.)*

**Answer:**

---

In [ ]:
# Train optimal model 1
num_epochs = ...
model_1_optimal = build_model_1()
history_1_optimal = model_1_optimal.fit(x_train, y_train, batch_size=128, epochs=num_epochs, validation_split=0.1)
plot_training_history(history_1_optimal, 1)

In [ ]:
# Evaluate optimal model 1

...

So far, you manually explored how many epochs are necessary to successfully train the model. However, Tensorflow gives you an option to automate this called [early stopping](https://keras.io/api/callbacks/early_stopping/). See also [here](https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/) and [here](https://towardsdatascience.com/a-practical-introduction-to-early-stopping-in-machine-learning-550ac88bc8fd).

---
**Task 7:** Use an early stopping callback in your fitting function to find the optimal number of epochs. Use reasonable configurations. How many epochs does it train for?

**Answer:**

---

In [ ]:
# Train model 1 with early stopping

early_stopping_cb = EarlyStopping(
    monitor=...,
    patience=...,
    min_delta=...,
    mode=...
)

num_epochs = 200
model_1_es = build_model_1()
history_1_es = model_1_es.fit(x_train, y_train, batch_size=128, epochs=num_epochs, validation_split=0.1, callbacks=[early_stopping_cb])
plot_training_history(history_1_es, 1)

---
**Task 8:** Explain the parameters you used for your early stopping callback. Why did you choose these?

**Answer:**

**Task 9:** Now train and evaluate models 2 and 3 below. How do they compare to model 1?

**Answer:**

**Task 10:** Create a bar plot comparing the models' accuracy. Have a look at [Matplotlib](https://matplotlib.org/stable/gallery/lines_bars_and_markers/bar_colors.html) or [Seaborn](https://seaborn.pydata.org/examples/grouped_barplot.html) for creating bar plots.

---

### Train models 2 and 3

### Evaluate models 2 and 3

In [ ]:
#@title Bar plot for task 10

...

---
**Task 11:** Briefly explain your plot of task 10.

**Answer:**

---

### On-device resource consumption

After training your three models we want to know whether we can run them on a microcontroller or whether they are too large. We will use the [Edge Impulse Python SDK](https://docs.edgeimpulse.com/docs/tools/edge-impulse-python-sdk) for profiling, so if you didn't add your API key on top, now is the time.

To start, we need to find the right target device for profiling. You are looking for the *Arduino Nano 33 BLE*.

In [ ]:
# List the available profile target devices
ei.model.list_profile_devices()

Next you can estimate the memory usage and inference time for each of your models.

In [ ]:
# Estimate the RAM, ROM, and inference time for our model on the target hardware family

your_model = ...
your_device = ...

try:
    profile = ei.model.profile(model=your_model,
                               device=your_device)
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

---
**Task 12:** Estimate the memory usage and inference time for each of your three final models. **Set your models' performance into perspective to their ROM and RAM usage and their inference time**. Please do **<ins>not</ins>** use a table for this, but plot it, for example with [Matplotlib](https://matplotlib.org/stable/gallery/lines_bars_and_markers/bar_colors.html) or [Seaborn](https://seaborn.pydata.org/examples/grouped_barplot.html). Bar plots should be a good option for it. On the x-axis you can list the model and on the y-axis, you can show the respective memory usage for ROM and RAM. *(You can reuse your code for plotting below in part 2.)*

**Task 13:** Briefly explain your plot(s) of task 12.

**Answer:**

---

## Part 2: Build your own model (FashionMNIST or CIFAR10)

In this part you apply your knowledge on classifying image data with neural networks. You can choose whether you want to use the FashionMNIST or the CIFAR10 dataset. Like MNIST both datasets contain images distributed over 10 classes. FashionMNIST should be the easier of the two and CIFAR10 the more challenging dataset.

Please choose **one** of the two datasets. If you want to deepen your knowledge on neural networks further, feel free to also try working with the other dataset. **Please mark which of the datasets you choose!**

### FashionMNIST dataset

The FashionMNIST dataset, is a dataset by Zalando similar to the MNIST one you used above. But instead of handwritten digits, the dataset contains 70.000 grayscale images of clothing items with a resolution of 28x28 pixels.

Here's an example of how the data looks (each class takes three-rows):

![Example of FashionMNIST images](https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/fashion-mnist-sprite.png)


Each clothing items in the dataset is assigned to one of the following labels:


| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

You can load and splic the data with

```python
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
```




### CIFAR10 dataset

The CIFAR10 dataset, is a labeled subset of the [80 million tiny images](http://people.csail.mit.edu/torralba/tinyimages/) dataset by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton. The CIFAR-10 dataset consists of 60000 colour images with a resolution of 32x32 pixels (input shape: (32, 32, 3)) in 10 classes, with 6000 images per class. The dataset contains 50.000 training images and 10.000 test images.

Here's an example of how the data looks:

![Example of CIFAR10 images](https://www.tensorflow.org/static/tutorials/images/cnn_files/output_K3PAELE2eSU9_0.png)


Each image in the dataset is assigned to one of the following labels:


| Label | Description |
| --- | --- |
| 0 | airplane |
| 1 | automobile |
| 2 | bird |
| 3 | cat |
| 4 | deer |
| 5 | dog |
| 6 | frog |
| 7 | horse |
| 8 | ship |
| 9 | truck |

You can load and split the data with

```python
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
```

### Tasks

---

**Task 14:**
Your task is to design and train neural networks that successfully classify the items in the dataset, and evaluate these models.

By successful we mean

- for FashionMNIST: each of your models has an accuracy of >85%
- for CIFAR10: at least one of your models has an accuracy of >70%

Please create at least **three (3)** neural network architectures covering the following categories:

- only fully-connected layers
- only convolutional layers as hidden layers
- a combination of both

---


Please keep in mind that your final model should fit onto a microcontroller, namely the *Arduino Nano 33 BLE Sense*. Therefore, use the *Edge Impulse Python SDK* to estimate the models sizes and try to build models that both perform well and fit onto the microcontroller.

---

**Hand in:**
In the notebook you hand in, you should show the models you designed, the training parameters you used, and the performance (accuracy) of your models. Moreover, **set your models' performance into perspective to their ROM and RAM  usage and their inference time**. Please do **<ins>not</ins>** use a table for this, but plot it, for example with [Matplotlib](https://matplotlib.org/stable/gallery/lines_bars_and_markers/bar_colors.html) or [Seaborn](https://seaborn.pydata.org/examples/grouped_barplot.html). Bar plots should be a good option for it. On the x-axis you can list the model and on the y-axis, you can show the respective memory usage for ROM and RAM. **Please briefly explain what one can see in your plots.**

---

### Prepare the Data

### Build the models

### Train the models

### Evaluate the models' performance